## Environment Setup

Place your conda yaml in the below section

## Place Imports 

Place all the imports that you are aware of here.

In [ ]:
import argparse
import os
import pickle
import mlflow
import xgboost
from smdebug import SaveConfig
from smdebug.xgboost import Hook
import mlflow.xgboost



## Utility Functions

Place the standalone functions in the below section

In [ ]:
def parse_args():

    parser = argparse.ArgumentParser()

    parser.add_argument("--max-depth", type=int, default=5)
    parser.add_argument("--eta", type=float, default=0.2)
    parser.add_argument("--gamma", type=int, default=4)
    parser.add_argument("--min-child-weight", type=int, default=6)
    parser.add_argument("--subsample", type=float, default=0.8)
    parser.add_argument("--silent", type=int, default=0)
    parser.add_argument("--objective", type=str, default="binary:logistic")
    parser.add_argument("--num-round", type=int, default=50)
    parser.add_argument("--smdebug-path", type=str, default=None)
    parser.add_argument("--smdebug-frequency", type=int, default=1)
    parser.add_argument("--smdebug-collections", type=str, default='metrics')
    parser.add_argument("--output-uri", type=str, default="/opt/ml/output/tensors",
                        help="S3 URI of the bucket where tensor data will be stored.")

    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR', '/opt/ml/model'))

    print('\n-------------- Environment Variables -------------\n')
    for key, value in os.environ.items():
        print('{}={}'.format(key, value))

    args = parser.parse_args()

    return args


In [ ]:
def create_smdebug_hook(out_dir, train_data=None, validation_data=None, frequency=1, collections=None,):

    save_config = SaveConfig(save_interval=frequency)
    hook = Hook(
        out_dir=out_dir,
        train_data=train_data,
        validation_data=validation_data,
        save_config=save_config,
        include_collections=collections,
    )

    return hook

## Data Cleaning

In [ ]:
def preprocessing(input_path):
    pass

## Visualize total data

Place all of the plots in the following section pertaining to the data

In [ ]:
# Test code

## Training Code

Place training code here

In [ ]:
def run_training():
    args = parse_args()
    # enable auto logging
    mlflow.xgboost.autolog()
    train, validation = args.train, args.validation
    parse_csv = "?format=csv&label_column=0"
    dtrain = xgboost.DMatrix(train+parse_csv)
    dval = xgboost.DMatrix(validation+parse_csv)

    # enable auto logging
    mlflow.xgboost.autolog()

    watchlist = [(dtrain, "train"), (dval, "validation")]
    with mlflow.start_run():
        params = {
            "max_depth": args.max_depth,
            "eta": args.eta,
            "gamma": args.gamma,
            "min_child_weight": args.min_child_weight,
            "subsample": args.subsample,
            "silent": args.silent,
            "objective": args.objective}

        # The output_uri is a the URI for the s3 bucket where the metrics will be
        # saved.
        output_uri = (
            args.smdebug_path
            if args.smdebug_path is not None
            else args.output_uri
        )

        collections = (
            args.smdebug_collections.split(',')
            if args.smdebug_collections is not None
            else None
        )

        hook = create_smdebug_hook(
            out_dir=output_uri,
            frequency=args.smdebug_frequency,
            collections=collections,
            train_data=dtrain,
            validation_data=dval,
        )

        model = xgboost.train(
            params=params,
            dtrain=dtrain,
            evals=watchlist,
            num_boost_round=args.num_round,
            callbacks=[hook])

        if not os.path.exists(args.model_dir):
            os.makedirs(args.model_dir)

        model_location = os.path.join(args.model_dir, 'xgboost-model')
        pickle.dump(model, open(model_location, 'wb'))

        mlflow.xgboost.log_model(model, 'model', registered_model_name='xgb-customer-churn')

## Run Training

In [ ]:
try:
    run_training()
except Exception as e:
    print("Unable to execute training", e)

In [ ]:
!head 'test-data/test_sample.csv'

![CodePipeline](codepipeline.png "Sample CodePipeline View")